Previsions of the Number Rescue Units to be Deployed by Traffic Acccidents

Initially we are ask to built and develop a prediction model through ML which would allow to predict the severity of a road accident based on the weather conditions experienced locally.

The question put forward is very interesting but not of great use for the emergency services. In this sense that usually the severity of road accident also determined of the location, the hour day and if we are dealing with a vehicule, a bicycle or a pedestrian.

We will have to study the relation of location, hour of the day and weather conditons to predict possible accidents.

But just having the probality of accidents is not good enough since it's very difficult or impossible to determine in advance on an alert how much resources to mobilise for the intervention,being ambulance crews, specialist emergency teams (Physicist Traumatologue, Driver Paramedic, Paramedic Traumatologue (In Belgium : MUG or Delta's)  and the number Operation Quarters and Chirugical teams, intensive care units to prepare and the number of beds to foreseen in the hospitals.

So we will have to study within an accident the relation between the kind of accident , the number of victims to be expected and the fort coming severity of injuries.

To built and develop our model we have the short hand registration of the Collisions in  Seattle over the time span of a year.

We are also handed the MetaData of the various attibutes of the  file to unable us to determinec which elements will be needed.

The needed elements:

OBJECTID:ESRI unique identifier, which can be used as index key
ADDRTYPE: Collision address type:
                        • Alley
                        • Block
                        • Intersection
SEVERITYCODE: A code that corresponds to the severity of the collision:
                                    • 3—fatality
                                    • 2b—serious injury
                                    • 2—injury
                                    • 1—prop damage
                                    • 0—unknown
                                    
COLLISIONTYPE:Collision type
PERSONCOUNT:The total number of people involved in the collision
PEDCOUNT: The number of pedestrians involved in the collision. 
PEDCYLCOUNT: The number of bicycles involved in the collision.
VEHCOUNT:The number of vehicles involved in the collision.
INJURIES:The number of total injuries in the collision.
SERIOUSINJURIES:The number of serious injuries in the collision.
FATALITIES: The number of fatalities in the collision.
WEATHER: A description of the weather conditions during the time of the collision.
ROADCOND: The condition of the road during the collision.

Let us load the datafile to check if we can find back all needed attributes and how the incidents are incodedin the various fields.




In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [2]:
!wget -O Data-Collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-08-20 17:26:39--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: 'Data-Collisions.csv'

Data-Collisions.csv 100%[===================>]  70.49M  3.19MB/s    in 10s     

2020-08-20 17:26:50 (7.08 MB/s) - 'Data-Collisions.csv' saved [73917638/73917638]



In [3]:
df_coll= pd.read_csv('Data-Collisions.csv')

/Users/marcvloeberghs/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
count= df_coll['ROADCOND'].isna().sum()
count

5012

In [5]:
df_coll['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [6]:
df_coll.dropna(subset=['OBJECTID'],inplace = True)

In [7]:
df_coll.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


Grazing through the datafile we noticed that some attribes are missing (INJURIES, SERIOUSINJURIES,FATALITIES). 

By analizing the 'SEVERITYCODE' we notice that it has been limited to a minimum compared to  the Metadata description.

This makes that we unforntunately have to reduce our scope of our initial project.

We will limit ourselves to the first part of our project being predicting the possibility of a road accident based on weather- , road conditions and location for Seattle.

The attributes we will be using:

OBJECTID:ESRI unique identifier, which can be used as index key
ADDRTYPE: Collision address type:
                        • Alley
                        • Block
                        • Intersection
SEVERITYCODE: A code that corresponds to the severity of the collision:
                                    • 3—fatality
                                    • 2b—serious injury
                                    • 2—injury
                                    • 1—prop damage
                                    • 0—unknown
LOCATION: Description of the general location of the collision.
WEATHER: A description of the weather conditions during the time of the collision.
ROADCOND: The condition of the road during the collision.


Lets us built a reduce dataframe which we will be using to built up our ML model.



In [8]:
df_sev = df_coll[['OBJECTID','SEVERITYCODE','WEATHER','ROADCOND','ADDRTYPE','LOCATION']]

In [9]:
df_sev.head()

,OBJECTID,SEVERITYCODE,WEATHER,ROADCOND,ADDRTYPE,LOCATION
0,1,2,Overcast,Wet,Intersection,5TH AVE NE AND NE 103RD ST
1,2,1,Raining,Wet,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N
2,3,1,Overcast,Dry,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST
3,4,1,Clear,Dry,Block,2ND AVE BETWEEN MARION ST AND MADISON ST
4,5,2,Raining,Wet,Intersection,SWIFT AVE S AND SWIFT AV OFF RP


In [10]:
df_sev['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [11]:
df_sev['ROADCOND'].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [12]:
df_sev['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

We notice that for ROADCOND and WEATHER some data fields are missing and replaced by 'Unknown'.
Let's replace 'Unknown' by 'nan' which will allows us to discard the rows with 'Unknown'.

In [13]:
df_sev.replace("Unknown", np.nan, inplace = True)

/Users/marcvloeberghs/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4166: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [14]:
df_sev['ROADCOND'].value_counts()

Dry               124510
Wet                47474
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [15]:
df_sev['WEATHER'].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

We set 'OBJECTID' as index field.

In [16]:
df_sev.set_index('OBJECTID')

,SEVERITYCODE,WEATHER,ROADCOND,ADDRTYPE,LOCATION
OBJECTID,,,,,
1,2,Overcast,Wet,Intersection,5TH AVE NE AND NE 103RD ST
2,1,Raining,Wet,Block,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N
3,1,Overcast,Dry,Block,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST
4,1,Clear,Dry,Block,2ND AVE BETWEEN MARION ST AND MADISON ST
5,2,Raining,Wet,Intersection,SWIFT AVE S AND SWIFT AV OFF RP
...,...,...,...,...,...
219543,2,Clear,Dry,Block,34TH AVE S BETWEEN S DAKOTA ST AND S GENESEE ST
219544,1,Raining,Wet,Block,AURORA AVE N BETWEEN N 85TH ST AND N 86TH ST
219545,2,Clear,Dry,Intersection,20TH AVE NE AND NE 75TH ST


In [17]:
df_sev.dtypes

OBJECTID         int64
SEVERITYCODE     int64
WEATHER         object
ROADCOND        object
ADDRTYPE        object
LOCATION        object
dtype: object